<a href="https://colab.research.google.com/github/Juanitorivi/UFV-visualizacion/blob/main/Ejercicios_de_clase/Clase18_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1º Número de actual y forecast

In [11]:
import pandas as pd

# Cargar el archivo CSV
file_path = '/content/datos_ejercicio_ventas.csv'
data = pd.read_csv(file_path, sep=",")

# Contar los valores en la columna 'SCENARIO'
scenario_counts = data['SCENARIO'].value_counts()

# Mostrar el resultado
print(scenario_counts)

SCENARIO
AI_forecast    17766
actual           900
Name: count, dtype: int64


2º Horizonte de predicción: es cuando haces una predicción, esta la haces para un número de pasos en el futuro , si estoy en enero y hago una predicción para febrero el horizonte de predicción es solo 1.

In [12]:
# Revisar el tipo de datos de las columnas
print(data.dtypes)

# Asegurarse de que los tipos sean consistentes
# Convertir la columna 'FORECAST_YEAR' a cadena si es necesario
data['FORECAST_YEAR'] = data['FORECAST_YEAR'].astype(str)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


# *Nº PAISES Y PRODUCTOS*

In [13]:
# Contar los valores en la columna 'COUNTRY'
paises_counts = data['COUNTRY'].value_counts()

# Mostrar el resultado
print(paises_counts)

COUNTRY
Great Britain    4008
Netherlands      2476
Hungary          2384
Denmark          2294
Spain            2152
Norway           2062
Czech            1403
Portugal         1179
Italy             708
Name: count, dtype: int64


Podemos observar que hay 9 países distintos

In [14]:
# Contar los valores en la columna 'COUNTRY'
productos_counts = data['SUBBRAND'].value_counts()

# Mostrar el resultado
print(productos_counts)

SUBBRAND
Pepsi Max (L3)        5598
Pepsi Regular (L3)    3636
7up (L3)              2694
Lipton (L3)           2693
7up Free (L3)         2343
Mountain Dew (L3)     1702
Name: count, dtype: int64


# **HORIZONTES DE PREDICCIÓN**


Para ver el horizonte de predicción, nos fijamos en un país y un producto (Portugal, Lipton)

In [15]:
portugal_lipton = data[(data['COUNTRY'] == 'Portugal') & (data['SUBBRAND'] == 'Lipton (L3)')]

# Mostrar las primeras filas del nuevo DataFrame
print(portugal_lipton.head())

      COUNTRY     SUBBRAND  YEAR  MONTH     SCENARIO FORECAST FORECAST_YEAR  \
0    Portugal  Lipton (L3)  2023     12  AI_forecast  AI_P02F        2023.0   
292  Portugal  Lipton (L3)  2023     10       actual      NaN           nan   
454  Portugal  Lipton (L3)  2024      9  AI_forecast  AI_P04F        2023.0   
490  Portugal  Lipton (L3)  2023      7  AI_forecast  AI_P04F        2023.0   
530  Portugal  Lipton (L3)  2023      9  AI_forecast  AI_P06F        2023.0   

           AMOUNT  
0    7.543562e+05  
292  1.206886e+06  
454  7.122028e+05  
490  8.897866e+05  
530  7.676745e+05  



Ahora eliminamos los valores de ACTUAL porque esos no son predicciones sino ventas

In [16]:
# Filtrar los datos para eliminar las filas donde el valor en 'SCENARIO' es 'actual'
portugal_lipton1 = portugal_lipton[portugal_lipton['SCENARIO'] != 'actual']

# Mostrar las primeras filas del nuevo DataFrame
print(portugal_lipton1.head())

      COUNTRY     SUBBRAND  YEAR  MONTH     SCENARIO FORECAST FORECAST_YEAR  \
0    Portugal  Lipton (L3)  2023     12  AI_forecast  AI_P02F        2023.0   
454  Portugal  Lipton (L3)  2024      9  AI_forecast  AI_P04F        2023.0   
490  Portugal  Lipton (L3)  2023      7  AI_forecast  AI_P04F        2023.0   
530  Portugal  Lipton (L3)  2023      9  AI_forecast  AI_P06F        2023.0   
690  Portugal  Lipton (L3)  2025      1  AI_forecast  AI_P09F        2023.0   

            AMOUNT  
0    754356.237194  
454  712202.846157  
490  889786.554294  
530  767674.504724  
690  790023.719723  


In [17]:
portugal_lipton_02 = portugal_lipton1[(portugal_lipton1['FORECAST'] == 'AI_P02F') & (portugal_lipton1['FORECAST_YEAR'] == '2023.0')]

# Mostrar las primeras filas del nuevo DataFrame
print(portugal_lipton_02.head())

       COUNTRY     SUBBRAND  YEAR  MONTH     SCENARIO FORECAST FORECAST_YEAR  \
0     Portugal  Lipton (L3)  2023     12  AI_forecast  AI_P02F        2023.0   
742   Portugal  Lipton (L3)  2023      9  AI_forecast  AI_P02F        2023.0   
953   Portugal  Lipton (L3)  2024      5  AI_forecast  AI_P02F        2023.0   
2311  Portugal  Lipton (L3)  2023      5  AI_forecast  AI_P02F        2023.0   
3353  Portugal  Lipton (L3)  2024      7  AI_forecast  AI_P02F        2023.0   

             AMOUNT  
0     754356.237194  
742   829026.131686  
953   668010.174306  
2311  738743.381537  
3353  903228.434649  


In [18]:
# Contar los valores en la columna 'SCENARIO'
portugal_counts = portugal_lipton_02['SCENARIO'].value_counts()

# Mostrar el resultado
print(portugal_counts)

SCENARIO
AI_forecast    18
Name: count, dtype: int64



Podemos observar que en enero de 2023 hemos realizado 18 puntos de predicciones

# **HISTORICO DE ACTUAL/FORECAST**

Histórico de ACTUALS es la primera y la última predicción de un producto

In [19]:
# Filtrar para obtener solo los escenarios actuales
historico_forecast = portugal_lipton[portugal_lipton['SCENARIO'] == 'actual']

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast['fecha_actual'] = pd.to_datetime(historico_forecast['YEAR'].astype(str) + '-' + historico_forecast['MONTH'].astype(str) + '-01')

# Encontrar la primera fecha solo con mes y año
primera_fecha_actual = historico_forecast['fecha_actual'].min().strftime('%Y-%m')

# Encontrar la última fecha solo con mes y año
ultima_fecha_actual = historico_forecast['fecha_actual'].max().strftime('%Y-%m')

# Mostrar los resultados
print("Primera fecha actual (Año-Mes):", primera_fecha_actual)
print("Última fecha actual (Año-Mes):", ultima_fecha_actual)



Primera fecha actual (Año-Mes): 2023-01
Última fecha actual (Año-Mes): 2024-08


<ipython-input-19-21c62ca49877>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_forecast['fecha_actual'] = pd.to_datetime(historico_forecast['YEAR'].astype(str) + '-' + historico_forecast['MONTH'].astype(str) + '-01')


Histórico de FORECAST

In [20]:
# Filtrar para obtener solo los escenarios actuales
historico_forecast_prim = portugal_lipton[portugal_lipton['FORECAST'] == 'AI_P02F']

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast_prim['fecha_actual'] = pd.to_datetime(historico_forecast_prim['YEAR'].astype(str) + '-' + historico_forecast_prim['MONTH'].astype(str) + '-01')

# Encontrar la primera fecha solo con mes y año
primera_fecha_prim = historico_forecast_prim['fecha_actual'].min().strftime('%Y-%m')


# Filtrar para obtener solo los escenarios actuales
historico_forecast_ult = portugal_lipton[portugal_lipton['FORECAST'] == 'AI_PF']

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast_ult['fecha_actual'] = pd.to_datetime(historico_forecast_ult['YEAR'].astype(str) + '-' + historico_forecast_ult['MONTH'].astype(str) + '-01')

# Encontrar la última fecha solo con mes y año
ultima_fecha_ultima = historico_forecast_ult['fecha_actual'].max().strftime('%Y-%m')

# Mostrar los resultados
print("Primera fecha actual (Año-Mes):", primera_fecha_prim)
print("Última fecha actual (Año-Mes):", ultima_fecha_ultima)

Primera fecha actual (Año-Mes): 2023-02
Última fecha actual (Año-Mes): 2024-06


<ipython-input-20-3ec805eae333>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_forecast_prim['fecha_actual'] = pd.to_datetime(historico_forecast_prim['YEAR'].astype(str) + '-' + historico_forecast_prim['MONTH'].astype(str) + '-01')
<ipython-input-20-3ec805eae333>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_forecast_ult['fecha_actual'] = pd.to_datetime(historico_forecast_ult['YEAR'].astype(str) + '-' + historico_forecast_ult['MONTH'].astype(str) + '-01')


In [24]:


# 1. Distribución de ventas por país
ventas_por_pais = data['COUNTRY'].value_counts()
print("Distribución de ventas por país:\n", ventas_por_pais)

# 2. Distribución de ventas por año y mes
ventas_por_mes_ano = data.groupby(['YEAR', 'MONTH']).size()
print("\nDistribución de ventas por año y mes:\n", ventas_por_mes_ano)

# 3. Distribución de ventas por marca (SUBBRAND)
ventas_por_marca = data['SUBBRAND'].value_counts()
print("\nDistribución de ventas por marca:\n", ventas_por_marca)

# 4. Tendencia y estacionalidad del país con menos ventas
pais_con_menos_ventas = ventas_por_pais.idxmin()
ventas_pais_menos = data[data['COUNTRY'] == pais_con_menos_ventas].groupby(['YEAR', 'MONTH']).size()
print(f"\nTendencia de ventas para {pais_con_menos_ventas}:\n", ventas_pais_menos)

# 5. Tendencia y estacionalidad de la marca con más ventas
marca_con_mas_ventas = ventas_por_marca.idxmax()
ventas_marca_mas = data[data['SUBBRAND'] == marca_con_mas_ventas].groupby(['YEAR', 'MONTH']).size()
print(f"\nTendencia de ventas para la marca {marca_con_mas_ventas}:\n", ventas_marca_mas)

# 6. Predicciones para España y análisis de su precisión
# Filtrar las predicciones hechas en España
predicciones_espana = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'AI_forecast')]
ventas_reales_espana = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'actual')]

# Comparar predicciones con ventas reales
predicciones_vs_reales = pd.merge(predicciones_espana, ventas_reales_espana, on=['YEAR', 'MONTH'], suffixes=('_pred', '_real'))
print("\nComparación de predicciones y ventas reales en España:\n", predicciones_vs_reales[['YEAR', 'MONTH', 'AMOUNT_pred', 'AMOUNT_real']])


Distribución de ventas por país:
 COUNTRY
Great Britain    4008
Netherlands      2476
Hungary          2384
Denmark          2294
Spain            2152
Norway           2062
Czech            1403
Portugal         1179
Italy             708
Name: count, dtype: int64

Distribución de ventas por año y mes:
 YEAR  MONTH
2023  1         115
      2         187
      3         258
      4         340
      5         423
      6         502
      7         584
      8         671
      9         757
      10        848
      11        938
      12       1032
2024  1        1032
      2        1033
      3        1032
      4        1033
      5        1031
      6        1032
      7         962
      8         890
      9         774
      10        692
      11        609
      12        530
2025  1         448
      2         361
      3         274
      4         184
      5          94
dtype: int64

Distribución de ventas por marca:
 SUBBRAND
Pepsi Max (L3)        5598
Pepsi Regular (L3